In [ ]:
import numpy as np
import pandas as pd
import keras
from keras import layers
from keras.models import Model
from keras.optimizers import SGD

df = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')


##preparation des données
    
    
    ##bedrooms
bedrooms = df[df.bedrooms > 10]
    ## supprimer les lignes
for row in bedrooms.index:
    df = df.drop([row])
    
    
    ##bathrooms
bathrooms = df[df.bathrooms > 6]
    ##supprimer ses lignes
for row in bathrooms.index:
    df = df.drop([row])

    
    ##price
price = df[df.price>3000000]
    ##supprimer les lignes
for row in price.index:
    df = df.drop([row])
    ##log price
df['price_log']= np.log(1+df['price'])


    ##sqft_lot
sqft_lot = df[df.sqft_lot>500000]
    ##supprimer les lignes
for row in sqft_lot.index:
    df = df.drop([row])
    ##log
df['sqft_lot_log']=np.log(1+df['sqft_lot'])


    ##sqft_living
sqft_living = df[df.sqft_living>7000]
    ##supprimer les lignes
for row in sqft_living.index:
    df = df.drop([row])
    
    
    ##creation des dummies pour les columns View et Condition
df = pd.get_dummies(df,columns=['view'])
df = pd.get_dummies(df,columns=['condition'])


    ##utilisant quelques variables
df = df [['bedrooms','bathrooms','floors','sqft_living','sqft_lot_log','price_log',
        'waterfront','view_0','view_1', 'view_2', 'view_3','view_4', 'condition_1',
        'condition_2','condition_3', 'condition_4','condition_5']]


      ##separer les donnes entrer train et test 
train_df=df[:17000]
test_df=df[17000:]
train_X = train_df[['bedrooms','bathrooms','floors','sqft_living','sqft_lot_log','waterfront','view_0','view_1', 'view_2', 'view_3',
                   'view_4', 'condition_1', 'condition_2','condition_3', 'condition_4',
                   'condition_5']].values 
train_Y = train_df['price_log'].values

test_X  = test_df[['bedrooms','bathrooms','floors','sqft_living','sqft_lot_log','waterfront','view_0','view_1', 'view_2', 'view_3',
                   'view_4', 'condition_1', 'condition_2','condition_3', 'condition_4',
                   'condition_5']].values 
test_Y  = test_df['price_log'].values

        ##min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)


        ##preparation de model
inpt   = layers.Input(shape=(16,))
x      = layers.Dense(300) (inpt)    
actv   = layers.Activation('relu') (x)
output = layers.Dense(1) (actv)
model  = Model(inpt,output)
model.summary()
opt = SGD()
model.compile(optimizer=opt,loss='mse') 
model.fit(train_X,train_Y,epochs=10)


        ##tester les prediction
predictions = model.predict(test_X)
predictions
np.exp(predictions)-1
